In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns 

## Models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

## Model evaluators
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import plot_roc_curve, plot_confusion_matrix

In [ ]:
df = pd.read_csv('/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv')
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df.dtypes

In [ ]:
sns.pairplot(df)

In [ ]:
plt.style.use('seaborn-pastel')
sns.countplot(x=df.output, hue=df.sex).set(title='Heart Disease Frequency by Sex')
plt.legend(["Female", "Male"]);

In [ ]:
sns.scatterplot(x=df.age, y=df.trtbps , hue=df.output);

In [ ]:
sns.histplot(df.age)

In [ ]:
# Create a new crosstab and base plot
pd.crosstab(df.cp, df.output).plot(kind="bar", 
                                   figsize=(10,6), 
                                   color=["lightblue", "salmon"])

# Add attributes to the plot to make it more readable
plt.title("Heart Disease Frequency Per Chest Pain Type")
plt.xlabel("Chest Pain Type")
plt.ylabel("Frequency")
plt.legend(["No Disease", "Disease"])
plt.xticks(rotation = 0);

In [ ]:
corr_matrix = df.corr()
plt.figure(figsize=(15, 10))
sns.heatmap(corr_matrix, 
            annot=True, 
            linewidths=0.5, 
            fmt= ".2f", 
            cmap="YlGnBu");

In [ ]:
# Put models in a dictionary
models = {"KNN": KNeighborsClassifier(),
          "Logistic Regression": LogisticRegression(), 
          "Random Forest": RandomForestClassifier()}

# Create function to fit and score models
def fit_and_score(models, X_train, X_test, y_train, y_test):
    """
    Fits and evaluates given machine learning models.
    models : a dict of different Scikit-Learn machine learning models
    X_train : training data
    X_test : testing data
    y_train : labels assosciated with training data
    y_test : labels assosciated with test data
    """
    # Random seed for reproducible results
    np.random.seed(42)
    # Make a list to keep model scores
    model_scores = {}
    # Loop through models
    for name, model in models.items():
        # Fit the model to the data
        model.fit(X_train, y_train)
        # Evaluate the model and append its score to model_scores
        model_scores[name] = model.score(X_test, y_test)
    return model_scores

In [ ]:
X = df.drop('output', axis=1)
y = df.output

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
model_scores = fit_and_score(models, X_train, X_test, y_train, y_test)

In [ ]:
model_compare = pd.DataFrame(model_scores, index=['accuracy'])
model_compare.T.plot.bar()

In [ ]:
# Different LogisticRegression hyperparameters
log_reg_grid = {"C": np.logspace(-4, 4, 20),
                #"solver": ["liblinear"]
               }

# Setup grid hyperparameter search for LogisticRegression
gs_log_reg = GridSearchCV(LogisticRegression(max_iter=10000),
                          param_grid=log_reg_grid,
                          cv=5,
                          verbose=True)

# Fit grid hyperparameter search model
gs_log_reg.fit(X_train, y_train);

In [ ]:
gs_log_reg.best_params_

In [ ]:
gs_log_reg.score(X_test, y_test)

In [ ]:
y_preds = gs_log_reg.predict(X_test)

In [ ]:
plot_roc_curve(gs_log_reg, X_test, y_test);

In [ ]:
plot_confusion_matrix(gs_log_reg, X_test, y_test)

In [ ]:
print(classification_report(y_test, y_preds))

In [ ]:
cross_val_score(gs_log_reg, X_train, y_train)

In [ ]:
# Helper function for plotting feature importance
def plot_features(columns, importances, n=20):
    df = (pd.DataFrame({"features": columns,
                        "feature_importance": importances})
          .sort_values("feature_importance", ascending=False)
          .reset_index(drop=True))
    
    sns.barplot(x="feature_importance",
                y="features",
                data=df[:n],
                orient="h"
                ).set(title='Feature Importance')

In [ ]:
plot_features(list(X_train.columns), gs_log_reg.best_estimator_.coef_[0])

In [ ]:
gs_log_reg.best_estimator_.coef_[0]

In [ ]:
features_dict = dict(zip(df.columns, list(gs_log_reg.best_estimator_.coef_))) 

In [ ]:
gs_log_reg = LogisticRegression(C=0.615848211066026, max_iter=10000)
cv_metrics = pd.DataFrame({
    'accuracy': cross_val_score(gs_log_reg, X_train, y_train, scoring='accuracy').mean(),
    'precision': cross_val_score(gs_log_reg, X_train, y_train, scoring='precision').mean() ,
    'recall': cross_val_score(gs_log_reg, X_train, y_train, scoring='recall').mean() ,
    'f1': cross_val_score(gs_log_reg, X_train, y_train, scoring='f1').mean() ,
}, index=['value'])
cv_metrics

In [ ]:
cv_metrics.T.plot.bar(legend=False)